In [356]:
import pandas as pd
import numpy as np
import string, re
import swifter
import pickle 

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import bigrams 
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer

from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')

import spacy
nlp = spacy.load("en_core_web_sm")
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

In [357]:
df = pd.read_csv("C:/Users/arsen/Healthylicious/data/cleaned/csv/combined_dataset_cleaned.csv")

In [358]:
df['ID'] = range(1, len(df) + 1)

In [359]:
cols = ['ID'] + [col for col in df.columns if col != 'ID']
df = df[cols]

print(df.head())

   ID               Category                                      Title  \
0   1            Main Course                   Swedish Meatballs Recipe   
1   2  Appetizer,Main Course  Baked Crispy Buffalo Chicken Wings Recipe   
2   3              Appetizer                           Guacamole Recipe   
3   4              Appetizer                       Perfect Queso Recipe   
4   5              Appetizer                 Buffalo Chicken Dip Recipe   

   Total Time                                    All Ingredients  \
0          30  1 pound ground beef, 2 tablespoons diced onion...   
1          60  6 pounds chicken wings (trimmed), 3 tablespoon...   
2           5  2 avocados (halved, seeded, and peeled), 1 tab...   
3           5  1/2 cup pepper jack cheese, 1 cup grated white...   
4          25  2 cups chicken breasts (cooked and shredded), ...   

                                   Ingredient Groups  \
0  IngredientGroup(ingredients=['1 pound ground b...   
1  IngredientGroup(ingredien

In [360]:
df['Rating Counts'] = np.random.randint(1, 1001, size=len(df))

In [361]:
print(df.columns.tolist())

['ID', 'Category', 'Title', 'Total Time', 'All Ingredients', 'Ingredient Groups', 'Instructions', 'Nutrition', 'Cuisine', 'Yields', 'Image', 'Ratings', 'Description', 'Status', 'Rating Counts']


In [362]:
cols = df.columns.tolist()  # Dapatkan daftar kolom
cols.insert(12, cols.pop(cols.index('Rating Counts')))  # Pindahkan 'Rating Counts' ke posisi ke-13
df = df[cols] 
print(df.columns.tolist())

['ID', 'Category', 'Title', 'Total Time', 'All Ingredients', 'Ingredient Groups', 'Instructions', 'Nutrition', 'Cuisine', 'Yields', 'Image', 'Ratings', 'Rating Counts', 'Description', 'Status']


In [363]:
df = df.set_index('ID')
df = df.dropna(subset=['Title','Rating Counts'])   

In [364]:
measure_words = ['bottle', 'bottles', 'box', 'boxes', 'bunch','bunches', 'bushel','bushels', 
                 'can', 'cans', 'container', 'c', 'cup', 'cups', 'carton', 'cartons', 'half',
                 'dash','dashes', 'drop','drops','fl', 'fl.', 'fluid','jar', 'jars','ounce','ounces','oz',
                 'g', 'gallon','gallons', 'glass','glasses', 'gram','grams','kg','kgs', 'lb','lbs', 'inch', 'inches'
                 'liter','liters', 'l', 'large', 'medium', 'ml','mls', 'package','pkg','small', 'to taste',
                 'pinch','pinches', 'pint','pints', 'pound','pounds', 'qt','qts', 'quart','quarts',
                 'scoop','scoops', 'sliced','slivered','stick','sticks', 'tablespoon','tablespoons',
                 'tbs','tbsp','tbsps', 'teaspoon','teaspoons','tsp','tsps','whole','cut into cube','cut into piece','fully'
                 ,'optional mix in','optional topping', 'package', 'piece','bags','ball','cut']

In [365]:
def clean_once(text):
    # Remove anything inside parentheses, including the parentheses themselves
    text = re.sub(r'\([^)]*\)', '', text)
    
    text = re.sub(r"confectioner'?s'", 'confectioners', text)
    text = re.sub(r"confectioner'?s", 'confectioners', text)
    text = re.sub(r"hershey'?s", 'hersheys', text)
    text = re.sub(r"hershey'?s'", 'hersheys', text)
    
    # Replace "half-and-half" with "halfandhalf"
    text = text.replace('half-and-half', '')
    text = text.replace('peanut butter chocolate chip cookie', 'peanut butter, chocolate chip, cookie')
    text = text.replace('the traditional decoration is ', '')
    text = text.replace('of sauce', '')
    text = text.replace('portabella', 'portobello')
    text = text.replace('portabello', 'portobello')
    text = text.replace('pears', 'pear')
    text = text.replace('halfandhalf', '')
    text = text.replace('at room temperature', '')
    text = text.replace('for the topping', '')
    text = text.replace('broken into piece', '')
    text = text.replace('breadcrumb', 'bread crumb')
    text = text.replace('spring mix salad green', 'spring salad green')
    text = text.replace('spring salad green mix', 'spring salad green')
    text = text.replace('tomatoes', 'tomato')
    text = text.replace('crabmeat', 'crab meat')
    text = text.replace('bleu', 'blue')
    text = text.replace('gruyère', 'gruyere')
    text = text.replace('bbq', 'barbeque')
    text = text.replace('store bought', '')
    text = text.replace('another bread such as sourdough' , 'sourdough')
    text = text.replace('frozen ' , '')

    
    # Remove fraction characters
    text = re.sub(r'[\u00BC-\u00BE\u2150-\u215E]', '', text)
    
    ingredlist = []
    for ingred in text.split(", "):
        # Remove measurements like "8-ounce"
        ingred = re.sub(r'\d+-ounce ', '', ingred)
        # Remove digits
        ingred = re.sub(r'\d', '', ingred)
        ingred = re.sub('\w*\d\w*', ' ', ingred)
        ingred = re.sub(r'\s+', ' ', ingred)  # Remove extra whitespace
        ingred = re.sub('[%s]' % re.escape(string.punctuation), ' ', ingred)
        ingred = ingred.replace('/', ' ')     # Remove slashes
        ingred = ingred.replace(')', '')      # Remove closing parenthesis
        ingred = ingred.replace('.', '')      # Remove dot
        ingred = ingred.replace('-', '')      # Remove hyphen
        ingred = ingred.replace('"', '')      # Remove hyphen
        ingred = ingred.replace("'", '')      # Remove hyphen
        ingred = ingred.replace("&", '')      # Remove hyphen
        ingred = ingred.replace(' and ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace('flour for', ', ')     # Replace " and " with ", "
        ingred = ingred.replace('of sauce', '')     # Replace " and " with ", "
        ingred = ingred.replace(' stuffed with ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace('another bread such as ' , '')     # Replace " and " with ", "
        ingred = ingred.replace(' about ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' with ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' roll ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' or ', ', ')     # Replace " and " with ", "
        ingred = ingred.replace(' in ', ', ')     # Replace " and " with ", "
        ingred = ' ' + ingred + ' '# Padding in case measure_word is first or last
        for measure_word in measure_words:    # Remove measure words
            ingred = ingred.replace(' ' + measure_word + ' ', ' ')
        ingred = ingred.replace(' or ', ', ') # Replace " or " with ", "
        ingred = ingred.strip().lower()       # Strip leading/trailing whitespace and convert to lowercase
        ingredlist.append(ingred)
        cleaned_text = ', '.join(ingredlist)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Ensure single spaces
        cleaned_text = re.sub(r'\b(\w+)\s*,\s*\1\b', r'\1', cleaned_text)  # Remove repeated words separated by commas
        cleaned_text = re.sub(r',\s*,', ',', cleaned_text)  # Remove extra commas
        cleaned_text = re.sub(r'\s*,\s*', ', ', cleaned_text.strip())  # Ensure proper comma spacing
    return cleaned_text

In [366]:
df['Ingredients'] = df['All Ingredients'].apply(clean_once)

In [367]:
df['All Ingredients'][199]

"1 1/4 cups water, 1/2 cup granulated sugar, 1 1/4 teaspoons active dry yeast, 2 eggs, 1 1/2 teaspoons kosher salt, 1 cup evaporated milk, 6 1/2 - 7 cups all-purpose flour, 1/4 cup shortening, oil, confectioners' sugar"

In [368]:
df['Ingredients'][199]

'water, granulated sugar, active dry yeast, eggs, kosher salt, evaporated milk, all purpose flour, shortening, oil, confectioners sugar'

In [369]:
def word_singularizer(word):
    nlp_word = nlp(word)[0]
    lemma = nlp_word.text
    if nlp_word.tag_ in {"NNS", "NNPS"}:
        lemma = nlp_word.lemma_
    return lemma

def text_singularizer(text):
    ingredlist = []
    for ingred in text.split(', '):
        new_list = []
        for word in ingred.split():
            new_list.append(word_singularizer(word))
        ingred = ' '.join(new_list)
        ingredlist.append(ingred)        
    return ', '.join(ingredlist)

In [370]:
df['Ingredients'] = df['Ingredients'].fillna('')

In [371]:
teststr = df['Ingredients'][172]

teststr

'cream cheese, butter, confectioners sugar, molasses, vanilla extract, ginger, ground cinnamon, toffee bits'

In [372]:
unwanted_phrases = ['red skinned','flour for','for the topping','royal blue gel paste ','quick cooking','quick cook','cut into cube','for griddle','cream of chicken soup cream of chicken soup of','cream of','condensed cream of','to drizzle','cut into','fat free ','on the cob','dripping from','easy blender','dressing of choice', 'each tortellini','store bought soup','equal size', 'm ms','for topping', 'for rolling cookie','broken into chunks','bulbs of', 'with juice', 'more for tops','box duncan hines','bone in','box duncan hine',
                    'egg shaped','about that','add in','gluten free','day glazed', 'package unflavored', 'candy coated','strawberry flavored','stuffed with','slices thin', 'thinly sliced', 'all purpose','boneless skinless', 'half-and-half', 'thick-cut', 'thick cut', 'finely grated','hard boiled', 'pieces cooked', 'for garnish','to garnish', 'to cover', 'zest of','the traditional decoration is','frozen','into cube','into', 'thawed',
                    'restaurant style','black bean black bean','optional mix in','optional topping', 'the best','round of','long grain', 'low sugar', 'low sodium','head of', 'a little for handling','another bread such as','oven cooked','cooked shredded smoked','hard cooked', 'grated fresh','strong brewed', 'for deep fried', 'if used', 'in water', 'equal size','a little for handling', 'for deep frying', 'of espresso powder','of sauce','of sugar', 'top round']
unwanted_words = ['cooked','cut','store bought','tablespoon','storebought','about', 'favourite', 'favorite','breading','cream of chicken soup of water','bulb','jellied','reserved','herbed','sized','unsulphured','marinated','rubbed','grilled','stewed','mixed','undrained', 'colored','shelled','toasted', 'chilled','glazed','coated','malted', 'xl sized', 'unbaked','unflavored','creamed', 'sauuted', 'overripe', 'ripe', 'spread','rolled', 'granulated', 'hot', 'packed', 'plain', 'thinly', 'dried', 'unsweetened', 'roasted', 'boneless', 'skinless', 'cubed', 'minced', 'crumbled', 'assorted', 'softened', 'sharp',  'boiling', 'rinsed', 'drained', 'crushed', 'popped', 'thin', 'light', 'raw', 'natural', 'oven-cooked', 
                 'deseeded','grapeseed', 'cooked', 'steamed','roughly','black eyed','shredded', 'smoked', 'homemade', 'reduced', 'mild',  'baked', 'seedless', 'new', 'old', 'melted', 'pieces', 'finely', 'twirl', 'boiled', 'lump', 'pulled', 'blackened', 'bag', 'sundried', 'prepared', 'whipped', 'dry', 'wet', 'uncooked', 'recipe','package','packet','pan','regular','pan','roll','s','loaf',
                 'frozen','pickled', 'grated','grilled', 'packed', 'large', 'small', 'medium', 'fresh', 'ground', 'diced', 'chopped', 'neutral', 'slider', 'sheet','boneless', 'breasts', 'breast', 'skinless', 'salted', 'filled', 'brewed','dried','shaved','refried', 'canned', 'mashed', 'evaporated', 'seasoned', 'refigerated', 'distilled', 'caramelized','to', 'xl','bags','ball', 'cup','jumbo','mini','of sauce','one'
                 'thawed', 'active','violet gel paste ','the traditional decoration is','sweet','round','ever','peeled', 'pillsbury','cap','cutlet','from', 'flaked',  'pasteurized','pittled', 'sweetened', 'seasoned','half','another bread such as', 'best', 'unsalted','thick','thin','with bone in', 'unpilled', 'fried', 'halved', 'canned', 'refrigerated', 'one','refried', 'preferred', 'slice', 'fashioned', 'quarted', 'quartered', 'sweetened', 'unpeeled', 'pitted', 'deveined','perfect','piece','pinch']


def remove_unwanted_words_and_phrases(text):
    # Remove unwanted phrases
    for phrase in unwanted_phrases:
        text = re.sub(r'\b' + re.escape(phrase) + r'\b', '', text)
    
    # Remove unwanted words
    words = text.split(', ')
    cleaned_words = []
    for word in words:
        tokens = word.split()
        cleaned_tokens = [token for token in tokens if token not in unwanted_words]
        cleaned_words.append(' '.join(cleaned_tokens))
    
    # Join cleaned words and handle any extra spaces or commas
    cleaned_text = ', '.join(cleaned_words).replace(' ,', ',').strip()
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = re.sub(r',\s*,', ',', cleaned_text)
    cleaned_text = re.sub(r'\s*,\s*', ', ', cleaned_text)
    
    return cleaned_text

In [373]:
print("Preprocessed:\n", teststr, '\n')
singularized_text = text_singularizer(teststr)
print("Singularized:\n", singularized_text, '\n')
unwanted_words_removed_text = remove_unwanted_words_and_phrases(singularized_text)
print("Unwanted Words and Phrases Removed:\n", unwanted_words_removed_text, '\n')

Preprocessed:
 cream cheese, butter, confectioners sugar, molasses, vanilla extract, ginger, ground cinnamon, toffee bits 

Singularized:
 cream cheese, butter, confectioner sugar, molasses, vanilla extract, ginger, ground cinnamon, toffee bit 

Unwanted Words and Phrases Removed:
 cream cheese, butter, confectioner sugar, molasses, vanilla extract, ginger, cinnamon, toffee bit 



In [374]:
df['Ingredients'] = df['Ingredients'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [375]:
  # Takes 2 min per 1000 rows
df['IngredientsProcessed'] = df['Ingredients'].apply(text_singularizer)# Takes 1 min per 1000 rows
df['IngredientsRemovedAdj'] = df['IngredientsProcessed'].apply(remove_unwanted_words_and_phrases)# Takes 1 min per 1000 rows
df.head()

,Category,Title,Total Time,All Ingredients,Ingredient Groups,Instructions,Nutrition,Cuisine,Yields,Image,Ratings,Rating Counts,Description,Status,Ingredients,IngredientsProcessed,IngredientsRemovedAdj
ID,,,,,,,,,,,,,,,,,
1,Main Course,Swedish Meatballs Recipe,30,"1 pound ground beef, 2 tablespoons diced onion...",IngredientGroup(ingredients=['1 pound ground b...,For the meatballs:\r\nMix together all ingredi...,"{'calories': '220 kcal', 'carbohydrateContent'...",American,6 servings,https://addapinch.com/wp-content/uploads/2022/...,4.95,533,The Best Swedish Meatballs feature tender meat...,Complete,"ground beef, diced onion, egg, stone house sea...","ground beef, diced onion, egg, stone house sea...","beef, onion, egg, stone house seasoning, allsp..."
2,"Appetizer,Main Course",Baked Crispy Buffalo Chicken Wings Recipe,60,"6 pounds chicken wings (trimmed), 3 tablespoon...",IngredientGroup(ingredients=['6 pounds chicken...,Preheat the oven to 425º F. Line a baking shee...,"{'servingSize': '0.25 pound', 'calories': '419...",American,8 servings,https://addapinch.com/wp-content/uploads/2018/...,5.00,941,Crispy Baked Buffalo Chicken Wings that are so...,Complete,"chicken wings, cornstarch, stone house seasoni...","chicken wing, cornstarch, stone house seasonin...","chicken wing, cornstarch, stone house seasonin..."
3,Appetizer,Guacamole Recipe,5,"2 avocados (halved, seeded, and peeled), 1 tab...",IngredientGroup(ingredients=['2 avocados (halv...,Mash avocado with a fork or potato masher in a...,"{'carbohydrateContent': '7 g', 'proteinContent...",Mexican,6 servings,https://addapinch.com/wp-content/uploads/2013/...,4.50,638,"This easy Guacamole recipe, made with fresh in...",Complete,"avocados, lime juice, sea salt, clove garlic, ...","avocados, lime juice, sea salt, clove garlic, ...","avocados, lime juice, sea salt, clove garlic, ..."
4,Appetizer,Perfect Queso Recipe,5,"1/2 cup pepper jack cheese, 1 cup grated white...",IngredientGroup(ingredients=['1/2 cup pepper j...,Stovetop Queso:\r\nAdd cheeses and half of the...,"{'calories': '534 kcal', 'carbohydrateContent'...",American,2 servings,https://addapinch.com/wp-content/uploads/2015/...,5.00,623,This easy and delicious Queso recipe is made w...,Complete,"pepper jack cheese, grated white american chee...","pepper jack cheese, grated white american chee...","pepper jack cheese, white american cheese, whi..."
5,Appetizer,Buffalo Chicken Dip Recipe,25,"2 cups chicken breasts (cooked and shredded), ...",IngredientGroup(ingredients=['2 cups chicken b...,Preheat oven to 350º F.\r\nMix together all in...,"{'calories': '106 kcal', 'proteinContent': '4 ...",American,12 servings,https://addapinch.com/wp-content/uploads/2024/...,5.00,629,Buffalo Chicken Dip Recipe is the best easy ap...,Complete,"chicken breasts, cream cheese, ranch dressing,...","chicken breast, cream cheese, ranch dressing, ...","chicken, cream cheese, ranch dressing, buffalo..."


In [376]:
def remove_duplicates(recipe):
    ingredients = recipe.split(', ')
    unique_ingredients = list(dict.fromkeys(ingredients))  # Remove duplicates while maintaining order
    return ', '.join(unique_ingredients)

# Apply the function to each row in the 'Ingredients' column
df['IngredientsRemovedAdj'] = df['IngredientsRemovedAdj'].apply(remove_duplicates)

In [377]:
test = df.IngredientsRemovedAdj[386]
tokens = RegexpTokenizer(r'\w+').tokenize(test)
print(test)
print(tokens)


flour, sugar, cocoa powder, baking powder, baking soda, kosher salt, espresso powder, milk, oil, egg, vanilla extract, water, vanilla buttercream frosting
['flour', 'sugar', 'cocoa', 'powder', 'baking', 'powder', 'baking', 'soda', 'kosher', 'salt', 'espresso', 'powder', 'milk', 'oil', 'egg', 'vanilla', 'extract', 'water', 'vanilla', 'buttercream', 'frosting']


In [378]:
def commatokenizer(text):
    return text.split(', ')

def get_nouns(text):
    tokens = RegexpTokenizer(r'\w+').tokenize(text)
    nouns = {'NN', 'NNS', 'NNP', 'NNPS', 'NOUN', 'PROPN', 'NE', 'NNE', 'NR'}
    nounlist = [token for token in tokens if nlp(token)[0].tag_ in nouns]
    return ', '.join(nounlist)

def mytokenizer(combinedlist):
    ingredlist = combinedlist[0].split(', ')
    nounlist = combinedlist[1].split(', ')
    bigramlist = []
    for ingred in ingredlist:
        bigrms = [bi for bi in bigrams(ingred.split())]
        for bi in bigrms:
            if (bi[0] in nounlist) or (bi[1] in nounlist):
                bigramlist.append(' '.join((bi[0], bi[1])))
    return ', '.join(bigramlist + nounlist)

In [379]:
teststr = df.IngredientsRemovedAdj[386]
nouns = get_nouns(teststr)
combined = [teststr, nouns]

print("Original Text:\n", teststr, '\n')
print("Nouns:\n", nouns, '\n')
print("My Tokenizer Output:\n", mytokenizer(combined), '\n')
print("Comma Tokenizer Output:\n", commatokenizer(mytokenizer(combined)))

Original Text:
 flour, sugar, cocoa powder, baking powder, baking soda, kosher salt, espresso powder, milk, oil, egg, vanilla extract, water, vanilla buttercream frosting 

Nouns:
 flour, sugar, cocoa, powder, powder, kosher, salt, espresso, powder, milk, oil, egg, vanilla, water, vanilla, buttercream 

My Tokenizer Output:
 cocoa powder, baking powder, kosher salt, espresso powder, vanilla extract, vanilla buttercream, buttercream frosting, flour, sugar, cocoa, powder, powder, kosher, salt, espresso, powder, milk, oil, egg, vanilla, water, vanilla, buttercream 

Comma Tokenizer Output:
 ['cocoa powder', 'baking powder', 'kosher salt', 'espresso powder', 'vanilla extract', 'vanilla buttercream', 'buttercream frosting', 'flour', 'sugar', 'cocoa', 'powder', 'powder', 'kosher', 'salt', 'espresso', 'powder', 'milk', 'oil', 'egg', 'vanilla', 'water', 'vanilla', 'buttercream']


In [380]:
def process_ingredients(row):
    final = remove_unwanted_words_and_phrases(row)
    nouns = get_nouns(final)
    combined = [final, nouns]
    return mytokenizer(combined)

In [381]:
df['TokenizedIngredients'] = df['IngredientsRemovedAdj'].apply(process_ingredients)

In [382]:
print(df[['IngredientsProcessed','TokenizedIngredients']].head())

                                 IngredientsProcessed  \
ID                                                      
1   ground beef, diced onion, egg, stone house sea...   
2   chicken wing, cornstarch, stone house seasonin...   
3   avocados, lime juice, sea salt, clove garlic, ...   
4   pepper jack cheese, grated white american chee...   
5   chicken breast, cream cheese, ranch dressing, ...   

                                 TokenizedIngredients  
ID                                                     
1   stone house, house seasoning, cracker crumb, o...  
2   chicken wing, stone house, house seasoning, bl...  
3   lime juice, sea salt, clove garlic, roma tomat...  
4   pepper jack, jack cheese, white american, amer...  
5   cream cheese, ranch dressing, cheddar cheese, ...  


In [383]:
df.to_csv('raw_combined_ingredients.csv', index=False)

In [384]:
df.to_excel('raw_combined_ingredients.xlsx', index=False, engine='openpyxl')

In [385]:
df.iloc[0].TokenizedIngredients

'stone house, house seasoning, cracker crumb, olive oil, beef stock, heavy cream, worcestershire sauce, beef, onion, egg, stone, house, allspice, nutmeg, bread, cracker, parsley, olive, oil, butter, flour, beef, stock, broth, milk, cream, worcestershire'

In [386]:
df0 = df.copy()
df1 = df.copy()

In [387]:
print(df[(df['Ratings'] >= 4) & (df['Rating Counts'] >= 4)]['Ratings'].count())


1291


In [388]:
df1 = df[(df['Ratings'] >= 4) & (df['Rating Counts'] >= 4)]

print(df[df.Title.str.contains('[Hh]ummus')].Title.count())

2


In [389]:
# STOP_WORDS = ['ground', 'fresh ground', 'powder', 'cream of', 'boneless', 'skinless', 'skinless chicken', 'half'
#              'butter or', 'juice', 'fresh lemon', 'rind', 'salt and', 'and pepper', 'boiling water', 'cold water',
#              'sauce', 'soy', 'chip', 'soda', 'oil', 'or vegetable', 'oil or', 'vegetable', 'seed',
#              'shredded cheddar', 'grated cheddar', 'jack', 'monterey', 'cheese or', 
#             'half', 'of chicken', 'shredded mozzarella', 'cottage', 'confectioner', 'unsalted butter', 'of chicken',
#             'condensed cream', 'package', 'package cream', 'whip', 'stalk celery', 'bay', 'leaf',
#             'slice', 'slice white', 'slice bacon', 'slice bread', 'slice white', 'creamy', 'butter or', 'or butter',
#             'salt', 'yeast or', 'pepper or', 'white', 'lowfat', 'skim', 'milk or', 'instant', 'whip', 'light', 
#             'light corn', 'flake', 'worcestershire', 'dijon', 'cream or', 'salt pepper', 'wheat', 'squeezed lemon',
#              ]

In [390]:
def user_tokenize(ingreds):
    ingreds = clean_once(ingreds)
    ingreds = text_singularizer(ingreds)
    ingreds = remove_unwanted_words_and_phrases(ingreds)
    nouns = get_nouns(ingreds)
    ingredscombined = [ingreds, nouns]
    ingredstokenized = mytokenizer(ingredscombined)
    return ingredstokenized

In [391]:
# Apply user_tokenize function to the Ingredients column
#df['TokenizedIngredients'] = df['Ingredients'].apply(lambda x: user_tokenize(x))
vectorizer = TfidfVectorizer(tokenizer=commatokenizer, min_df=7, max_df=0.4)
docs = df['TokenizedIngredients']
doc_word = vectorizer.fit_transform(docs)

print(doc_word.shape)

(1295, 315)


c:\Users\arsen\Healthylicious\.venv\lib\site-packages\sklearn\feature_extraction\text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [392]:
nmf_model = NMF(20, random_state=10, max_iter=1000)
doc_topic = nmf_model.fit_transform(doc_word)
topic_word = nmf_model.components_

In [393]:
from sklearn.decomposition import NMF

# Experiment with different numbers of topics
n_topics = [15, 20, 25, 30]
best_nmf_model = None
best_score = float('inf')

for n in n_topics:
    nmf_model = NMF(n, random_state=10, max_iter=2000)
    doc_topic = nmf_model.fit_transform(doc_word)
    topic_word = nmf_model.components_
    score = nmf_model.reconstruction_err_
    
    if score < best_score:
        best_score = score
        best_nmf_model = nmf_model

nmf_model = best_nmf_model
doc_topic = nmf_model.fit_transform(doc_word)
topic_word = nmf_model.components_

def display_topics(model, feature_names, num_top_words, topic_names=None):
    for idx, topic in enumerate(model.components_):
        if not topic_names or not topic_names[idx]:
            print("\nTopic ", idx)
        else:
            print("\nTopic: '", topic_names[idx], "'")
        print(", ".join([feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]))

display_topics(nmf_model, vectorizer.get_feature_names_out(), 8)



Topic  0
egg, baking powder, flour, vanilla, vanilla extract, powder, sugar, butter

Topic  1
olive, olive oil, oil, zucchini, pork, oil spray, green bean, chop

Topic  2
stone, stone house, house seasoning, house, parsley, ranch, ranch seasoning, chili powder

Topic  3
cheddar, cheddar cheese, cheese, cream, cream cheese, bacon, sour cream, green onion

Topic  4
chocolate, chocolate chip, chip, coconut, coconut oil, sea salt, sea, candy

Topic  5
lemon, lemon juice, juice, lemon zest, zest, sugar, celery, mustard

Topic  6
powder, cocoa, cocoa powder, espresso, espresso powder, milk, vanilla extract, chili powder

Topic  7
red, red onion, onion, red wine, wine, vinaigrette, wine vinegar, dijon

Topic  8
brown, brown sugar, sugar, cinnamon, oat, vanilla, vanilla extract, sea salt

Topic  9
garlic, cloves garlic, clove garlic, onion, cumin, parsley, butter, tomato

Topic  10
self, self rising, rising flour, buttermilk, flour, cornmeal, butter, egg

Topic  11
parmesan, parmesan cheese, 

In [394]:
# df['All Ingredients'] = df['All Ingredients'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
# test = df.iloc[1]['Ingredients']
# print(test,'\n')
# print(user_tokenize(clean_once(test)),'\n')

In [395]:
# def clean_raw(text):
#     text = text["All Ingredients"]
#     ingredlist = []
#     for ingred in  text.split("#item,"):
#         ingred = ingred.replace('#item','')         # Scraping artifact on last item
#         ingredlist.append(ingred.strip())
#         ingredlist = [ingred for ingred in ingredlist if not ingred.isupper()]  # Important for multi-part recipes
#     return ingredlist

In [396]:
# test = df.iloc[np.random.choice(range(1,1295))]
# print(test.Title)
# clean_raw(test)
# test.TokenizedIngredients.split(', ')

In [397]:

key_ingredients_weights = {
    'beef': 10,
    'chicken': 8,
    'shrimp': 9,
    # Add more key ingredients and their specific weights
}


In [398]:
useringreds = "garlic, butter, cheese"
usertokens = user_tokenize(useringreds)
print('User Input: ', useringreds)
print('Tokens Generated: ', usertokens, '\n')

# Vectorize user input
user_vec = vectorizer.transform([usertokens])

# Adjust weights for key ingredients
feature_names = vectorizer.get_feature_names_out()
for ingredient, weight in key_ingredients_weights.items():
    if ingredient in usertokens:
        index = feature_names.tolist().index(ingredient)
        user_vec[0, index] *= weight

# Transform user vector into topic space
topic_vec = nmf_model.transform(user_vec)

# Compute similarity and get recommendations
indices = pairwise_distances(topic_vec, doc_topic, metric='cosine').argsort().ravel()

for index in indices[0:5]:
    print(df.iloc[index].Title.upper())
    print(df.iloc[index]["All Ingredients"], '\n')

User Input:  garlic, butter, cheese
Tokens Generated:  garlic, butter, cheese 

SMOKED LOBSTER TAILS RECIPE
4-6 lobster tails, 1/4 cup butter, 4 cloves garlic 

GARLIC CHEESE BISCUITS RECIPE
1 Cheddar Biscuits recipe, 1/4 teaspoon garlic powder, For the topping:, 3 tablespoons butter (melted), 1/2 teaspoon garlic powder 

GARLIC CHEESE BISCUITS RECIPE
1 Cheddar Biscuits recipe, 1/4 teaspoon garlic powder, For the topping:, 3 tablespoons butter (melted), 1/2 teaspoon garlic powder 

CORNISH HEN RECIPE
4 1 - 1 1/2 pounds Cornish hens (, rinsed), 4 cloves garlic (peeled), 4 shallots (peeled), 4 sprigs fresh rosemary, 6-8 sprigs fresh thyme, 4-6 tablespoons butter (softened), salt and pepper (to taste) 

BUFFALO CHICKEN LETTUCE WRAPS RECIPE
olive oil, 2 skinless (boneless chicken breasts, chopped into 1-inch pieces), 1/2 onion (diced), 2 cloves garlic (diced), 1/2 cup mushrooms (diced), 1/4 cup buffalo sauce, 12 lettuce leaves (such as iceberg or bibb), 1 celery (thinly sliced into 2-inch 

In [399]:
import pandas as pd

# Assume df is your existing DataFrame
# Extract the 'Ingredients' column
just_ingredients = df[['IngredientsRemovedAdj']]

# Save to a new CSV file
just_ingredients.to_csv('just_ingredients.csv', index=False)

print("Ingredients column has been saved to just_ingredients.csv")


Ingredients column has been saved to just_ingredients.csv


In [400]:
import pandas as pd

# Load the dataset
df = pd.read_csv('just_ingredients.csv')

# Extract all ingredients from all rows into a single list
all_ingredients = []
for ingredients in df['IngredientsRemovedAdj']:
    all_ingredients.extend(ingredients.split(', '))

# Remove duplicates and sort the list
unique_ingredients = sorted(set(all_ingredients))

# Create a new DataFrame from the unique ingredients
df_unique_ingredients = pd.DataFrame(unique_ingredients, columns=['Ingredient'])

# Save the new DataFrame to a CSV file
df_unique_ingredients.to_csv('UniqueIngredients.csv', index=False)

print("Unique ingredients have been saved to UniqueIngredients.csv")


Unique ingredients have been saved to UniqueIngredients.csv


In [401]:
# Example user input
useringreds = "butter, beef, salt"
usertokens = user_tokenize(useringreds)
print('User Input: ', useringreds)
print('Tokens Generated: ', usertokens, '\n')

# Increase weights of user ingredients
user_vec = vectorizer.transform([usertokens])
user_vec *= 10  # Increase weight

# Transform user vector into topic space
topic_vec = nmf_model.transform(user_vec)

# Compute similarity and get recommendations
indices = pairwise_distances(topic_vec, doc_topic, metric='cosine').argsort().ravel()

for index in indices[0:5]:
    print(df.iloc[index].Title.upper())
    print(df.iloc[index]["All Ingredients"], '\n')

User Input:  butter, beef, salt
Tokens Generated:  butter, beef, salt 



AttributeError: 'Series' object has no attribute 'Title'

In [ ]:
df["IngredientsCombined"][5]

['chicken breast, cream cheese, ranch dressing or blue cheese dressing, homemade or storebought, buffalo sauce recipe, grated cheddar cheese, stone house seasoning, bleu cheese crumble',
 'chicken, breast, cream, cheese, ranch, cheese, homemade, recipe, cheddar, cheese, stone, house, bleu, cheese, crumble']

In [ ]:
data = []

# Initialize a counter for the unique ID
unique_id = 1

# Iterate through all rows in the 'IngredientsTokenized' column
for index, row in df.iterrows():
    combined_ingredients = row["IngredientsCombined"]
    for ingredient_list in combined_ingredients:
        ingredients = ingredient_list.split(', ')
        for ingredient in ingredients:
            data.append({'ID': unique_id, 'Ingredient': ingredient})
            unique_id += 1

# Create a new DataFrame from the list of ingredients and their IDs
combined_ingredients_df = pd.DataFrame(data)

In [ ]:
combined_ingredients_df

,ID,Ingredient
0,1,ground beef
1,2,diced onion
2,3,egg
3,4,stone house seasoning
4,5,ground allspice
...,...,...
26817,26818,sugar
26818,26819,karo
26819,26820,kosher
26820,26821,salt


In [ ]:
combined_ingredients_df.to_csv('combined_ingredients.csv', index=False)